In [1]:
import pandas as pd
import numpy as np
import csv
# from surprise import SVD, SVDpp
# from surprise import Dataset
# import pandas as pd
# import surprise
# from surprise import Reader
# from surprise.model_selection import cross_validate
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import mean_squared_error
import lightgbm as lgbm
import copy
from sklearn.metrics import roc_auc_score

from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import GridSearchCV

In [2]:
train_df = pd.read_csv('Data/my_data/final_data/train3.csv')

C:\Users\yamsharon102\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
test_df = pd.read_csv('Data/my_data/final_data/test3.csv')

In [4]:
new_columns = []
for col in train_df.columns:
    if col in ['user', 'song', 'title' ,'artist', 'lyrics']:
        continue
    new_columns.append(col)
    if col == 'user_count':
        break
for col in train_df.columns:
    if 'Doc' in col or 'sentiment' in col:
        new_columns.append(col)
        

In [5]:
df_train = train_df[new_columns]
df_test = test_df[new_columns]
df_all = pd.concat([df_train, df_test])

In [6]:
df_all.head()

,age_diff,exp_DM,exp_MY,exp_Y,exp_YMD,reg_DM,reg_MY,reg_Y,reg_YMD,s_country,...,u_age,user_count,Doc2Vec_0,Doc2Vec_1,Doc2Vec_2,Doc2Vec_3,Doc2Vec_4,sentiment_negative,sentiment_neutral,sentiment_positive
0,2.0,510.0,102017.0,2017.0,20171005.0,201.0,12012.0,2012.0,20120102.0,GB,...,0.0,1287,0.087046,-0.913331,-1.798138,0.423479,0.183180,0.224,0.621,0.156
1,2.0,510.0,102017.0,2017.0,20171005.0,201.0,12012.0,2012.0,20120102.0,US,...,0.0,1287,1.400352,-2.319849,-2.563501,1.098209,-1.523817,0.140,0.773,0.088
2,21.0,403.0,32018.0,2018.0,20180304.0,711.0,112011.0,2011.0,20111107.0,US,...,26.0,247,-4.439262,-4.166830,-1.815961,2.514746,2.728830,0.148,0.816,0.035
3,10.0,3009.0,92017.0,2017.0,20170930.0,2012.0,122015.0,2015.0,20151220.0,GB,...,18.0,84,0.416141,-0.768392,-2.176152,1.724006,-1.827583,0.042,0.614,0.343
4,6.0,3009.0,92017.0,2017.0,20170930.0,2012.0,122015.0,2015.0,20151220.0,GB,...,18.0,84,-0.113220,-0.696612,-1.224584,-0.071810,0.138876,0.076,0.712,0.213


In [7]:
ua_svd = df_all['svd_ua']
ua_svd_fixed = []
for x in ua_svd:
    try:
        ua_svd_fixed.append(float(x))
    except:
        ua_svd_fixed.append(float(x[2:-2]))
df_all['svd_ua'] = ua_svd_fixed

In [8]:
if 's_country' in df_all.columns:
    df_all['country'] = pd.factorize(df_all['s_country'])[0] + 1
    df_all = df_all[[i for i in list(df_all.columns) if 's_country' != i]]
X = df_all[[x for x in df_all.columns if 'sim' not in x and 'count' not in x and
            'svd' not in x and x not in ['user', 'song', 'title' ,'artist', 'lyrics', 'target']]]
Y = df_all['target']

In [9]:
X.head()

,age_diff,exp_DM,exp_MY,exp_Y,exp_YMD,reg_DM,reg_MY,reg_Y,reg_YMD,s_year,u_age,Doc2Vec_0,Doc2Vec_1,Doc2Vec_2,Doc2Vec_3,Doc2Vec_4,sentiment_negative,sentiment_neutral,sentiment_positive
0,2.0,510.0,102017.0,2017.0,20171005.0,201.0,12012.0,2012.0,20120102.0,2016.0,0.0,0.087046,-0.913331,-1.798138,0.423479,0.183180,0.224,0.621,0.156
1,2.0,510.0,102017.0,2017.0,20171005.0,201.0,12012.0,2012.0,20120102.0,2016.0,0.0,1.400352,-2.319849,-2.563501,1.098209,-1.523817,0.140,0.773,0.088
2,21.0,403.0,32018.0,2018.0,20180304.0,711.0,112011.0,2011.0,20111107.0,2013.0,26.0,-4.439262,-4.166830,-1.815961,2.514746,2.728830,0.148,0.816,0.035
3,10.0,3009.0,92017.0,2017.0,20170930.0,2012.0,122015.0,2015.0,20151220.0,2010.0,18.0,0.416141,-0.768392,-2.176152,1.724006,-1.827583,0.042,0.614,0.343
4,6.0,3009.0,92017.0,2017.0,20170930.0,2012.0,122015.0,2015.0,20151220.0,2006.0,18.0,-0.113220,-0.696612,-1.224584,-0.071810,0.138876,0.076,0.712,0.213


In [10]:
train_set_x = X.head(len(train_df))
train_set_y = Y.head(len(train_df))
test_set_x = X.tail(len(test_df))
test_set_y = Y.tail(len(test_df))

In [11]:
model1 = lgbm.LGBMClassifier(learning_rate=0.05, max_depth=15, 
                             num_leaves=2**8, colsample_bytree=0.7,
                             subsample=0.7)
model1.fit(train_set_x, train_set_y, verbose=1)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.7,
        importance_type='split', learning_rate=0.05, max_depth=15,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=256, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=0.7, subsample_for_bin=200000, subsample_freq=0)

In [12]:
model2 = lgbm.LGBMClassifier(learning_rate=0.1, max_depth=15, 
                             num_leaves=2**8, colsample_bytree=0.9,
                             subsample=0.9)
model2.fit(train_set_x, train_set_y, verbose=1)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.9,
        importance_type='split', learning_rate=0.1, max_depth=15,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=256, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=0.9, subsample_for_bin=200000, subsample_freq=0)

In [19]:
y1 = [x[1] for x in model1.predict_proba(train_set_x)]
print('model1 train AUC',roc_auc_score(train_set_y, y1))
y2 = [x[1] for x in model2.predict_proba(train_set_x)]
print('model2 train AUC',roc_auc_score(train_set_y, y2))
ensemble_train = [(y1[i]+y2[i])/2 for i in range(len(y1))]
print('ensemble_train train AUC',roc_auc_score(train_set_y, ensemble_train))

model1 train AUC 0.7181710621931445
model2 train AUC 0.7467266386761687
ensemble_train train AUC 0.7362984393859147


In [18]:

y_pred1 = [x[1] for x in model1.predict_proba(test_set_x)]
print('model1 test AUC',roc_auc_score(test_set_y, y_pred1))
y_pred2 = [x[1] for x in model2.predict_proba(test_set_x)]
print('model2 test AUC',roc_auc_score(test_set_y, y_pred2))
ensemble_test = [(y_pred1[i]+y_pred2[i])/2 for i in range(len(y_pred1))]
print('ensemble_test test AUC',roc_auc_score(test_set_y, ensemble_test))


model1 test AUC 0.6143656216242391
model2 test AUC 0.6079652283200468
model3 test AUC 0.6148584870111165
ensemble_test test AUC 0.6130332809143957
